In [1]:
from selenium import webdriver
options = webdriver.ChromeOptions() 
options.add_argument('--headless') 
options.add_argument('--no-sandbox') 
options.add_argument('--disable-dev-shm-usage') 
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', None)
import urllib.request
import urllib.parse
from urllib.parse import quote
import time
import datetime

# 동아일보도 San Francisco Chronicle 과 비슷하게 url에 날짜설정이 안되어서, 
# 날짜순서대로 정렬한후, if문을 돌려서 중간에 코드를 break 하는 형식으로 작성했습니다.
# San Francisco Chronicle 과 같게 chromedriver를 사용하면 시간이 너무 오래걸려(다량의 기사필요)
# urllib를 사용하였습니다. 또한, 진행중 DDOS공격으로 의심되어 IP밴당하는 사례가 생겨서,
# 추가적으로 밑에서 코드를 수정하였습니다.

In [2]:


def start_crawling(keyword,START_DATE,END_DATE,for_DDOS_pagenum): # 시작하는 함수입니다. keyword만 url에 들어가있고, 날짜는 추가적인 코드를 통해 거를 예정입니다.
    URL = f'ttps://www.donga.com/news/search?p=1&query={keyword}&check_news=1&more=1&sorting=1&search_date=1&v1=&v2=&range=1'
    start_date = datetime.datetime.strptime(START_DATE,'%Y-%m-%d')
    end_date = datetime.datetime.strptime(END_DATE,'%Y-%m-%d')
    get_link_from_news_title(URL,start_date,end_date,for_DDOS_pagenum)
            
def get_link_from_news_title(hURL,START_DATE,END_DATE,for_DDOS_pagenum):
    firstURL = 'h' + hURL # 경향과 마찬가지로 하이퍼링크 이슈에관한 디버깅입니다.
    req = urllib.request.Request(firstURL, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'})        
    source_code_from_url = urllib.request.urlopen(req)
    firstsoup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8') # 첫 화면 soup 따주고
    num1 = firstsoup.select('#content > div.searchContWrap > div.searchCont > h2 > span:nth-child(1)')[0].get_text()
    num2 = num1.replace('(총 ', '')
    num = int(num2.replace(' 건 검색)', ''))
    page_num = int(num /15) +1 # 경향과 마찬가지로 page_num을 구해줍니다.
    
    Flag = True # for문 break를 위한 Flag입니다.
    pagenum1 = for_DDOS_pagenum*100 # DDOS로 의심받기때문에, timesleep을 위해 추가하였습니다.
    pagenum2 = (for_DDOS_pagenum*100) +100 # 100page마다 150초씩 쉬어줍니다.
    if pagenum2 > page_num: # 또한 마지막에서 range에러가 날 수 있기때문에
        pagenum3 = page_num # if문으로 조건을 지정해서 range의 최댓값이 page_num보다 클수 없게 지정했습니다.
    else:
        pagenum3 = pagenum2
        DDOS = False # 실행할때 while True: 문을 쓰기때문에 그것을 break해줄 DDOS 플래그입니다.
        
    for j in range(pagenum1,pagenum3): # 0~100, 100~200, 등등 올라갑니다. 마지막엔 700~777(page_num)
        article_number = (j*15) +1 # 페이지당 15개입니다.
        URL = f'ttps://www.donga.com/news/search?p={article_number}&query={keyword}&check_news=1&more=1&sorting=1&search_date=1&v1=&v2=&range=1' 
        # 특이하게 url에서 p= 다음에 페이지 수가 아닌 기사갯수가 들어가더라구요 15 * i +1 의 형식입니다.
        req = urllib.request.Request('h'+URL, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'})        
        source_code_from_url = urllib.request.urlopen(req)
        soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8') # soup 따주고
        for i in range(0,len(soup.select('p.tit'))):        
            date1 = soup.select(f'#content > div.searchContWrap > div.searchCont > div:nth-child({i+2}) > div.t > p.tit > span')[0]
            date2 = date1.get_text()[:10] # date 따줍니다.
            try: # 가끔 date를 못가져 오는 경우가있는데, 이것은 바로 전의 기사의 날짜로 대체하는 try except문입니다.
                 # 날짜순으로 정렬을 했기때문에, 날짜가 변하는일은 거의 없을겁니다. (에러도 정말 가끔 일어납니다.)
                date = datetime.datetime.strptime(date2,'%Y-%m-%d')
            except:
                date = lastdate
            lastdate = date            
            if START_DATE >= date: # 날짜순으로 정렬된 기사를 거르는 if문입니다.
                if date >= END_DATE: 
                    str_date = str(date)[:10] # date를 str로 변환해주고 년,월,일만 남기고 잘라줍니다.
                    title_name = soup.select(f'#content > div.searchContWrap > div.searchCont > div:nth-child({i+2}) > div.t > p.tit > a:nth-child(1)')[0].get_text()
                    # 기사제목
                    title_link = soup.select(f'#content > div.searchContWrap > div.searchCont > div:nth-child({i+2}) > div.t > p.tit > a:nth-child(1)')[0]['href']
                    # 기사링크 따준후
                    TITLE_OF_ARTICLE.append(title_name) 
                    DATE_OF_ARTICLE.append(str_date)
                    LINK_OF_ARTICLE.append(title_link)
                    # 어펜드 해준후에 get_text로 넘겨줍니다
                    get_text(title_link)
                else:
                    Flag = False # for문의 break를 위한 Flag입니다.
                    break # 2010.01.01 이후 날짜는 가져올필요가없기에 break 해줍니다.
        
        print(f'{j+1}페이지 완료') 
        
        if Flag == False:
            break # 2010.01.01 이후 날짜는 가져올필요가없기에 break 해줍니다.
            
def get_text(link):
    req = urllib.request.Request(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'})        
    source_code_from_url = urllib.request.urlopen(req)
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')
    # 링크 받아서 soup 따줍니다
    
    # 전처리 과정인데, 동아일보는 기사 본문안에 부제목, 사진설명, 관련기사, 등등이 모여있어서 
    # 만약 있으면, 제거해주는 코드입니다.
    text = str(soup.find_all("div", {"class": "article_txt"})[0].get_text())# 전체 기사를 모아주고
    
    text1 = soup.find_all("div", {"class": "articlePhotoC"}) # 전체 기사중에서 사진설명을 모두 없애줍니다.
    for i in text1: #사진설명이 없을경우 text1이 빈 리스트이기때문에 실행되지 않습니다.
        x = str(i.get_text())
        text = text.replace(f'{x}', '')
    
    try: #관련기사가 없을경우 에러가 뜨기때문에 try except문 사용해서 관련기사가 없으면 실행 안됩니다.
        text2 = str(soup.find_all("div", {"class": "article_relation"})[0].get_text()) # 관련기사를 없애줍니다
        text = text.replace(f'{text2}', '')
    except:
        pass
    
    try: #마찬가지로 부제목 없애주는 함수입니다.
        text3 = soup.find_all("strong", {"sub_title"})[0].get_text() # 부제목 없애줍니다.
        text = text.replace(f'{text3}', '')
    except:
        pass
    
    text = text.replace('\n', '.') # 불필요한 Enter키는 우선 "."으로 바꿔줍니다.(후에 전처리)
    
    # 여기부턴 기사 본문 뒤에서 기자 이름등 쓸모없는 부분을 전처리 하는 부분인데, 
    # 이러한 순서대로 전처리를 하게되면 거의 대부분 없어지는것을 확인하였습니다.
    # find함수는 찾지못하면 -1이 return되기때문에, -1이 아니면 indexing 된것으로 보고
    # 전처리 진행하였습니다.
    # 다수를 위해 소수의 기사는 희생되었을 수도 있습니다....(TT)
    
    q = text.find('동아닷컴 기자')
    # 기사 본문에서 '동아닷컴 기자' 를 찾아서 그앞에 -4만큼 
    # /여기/(홍길동 동아닷컴 기자) 부터 뒤를 전부 없애주는 함수입니다.
    if q != -1:
        text = text[:q-4]

    m = text.find('동아닷컴')
    # 마찬가지로 동아닷컴이 있으면 그 뒤로는 전부 없애줍니다.
    # ex) 기사본문. /여기/(동아닷컴) 홍길동 기자
    
    # 거의 대부분의 기사가 동아닷컴 기자, 동아닷컴에서 전처리가 완료됩니다. 이후(기자, (서울=) 등은 앞의 전처리 과정에서 삭제됩니다.)
    # 간혹 삭제되지 않는 기사가 있기에 추가하였습니다.
    
    #마찬가지로 이러한 단어가 있으면 뒤에는 전부 제거해줍니다.
    if k != -1:
        text = text[:k-4]

    l = text.find('[서울=')
    if l != -1:
        text = text[:l]

    o = text.find('(서울=')
    if o != -1:
        text = text[:o]

    p = text.find('도쿄=')
    if p != -1:
        text = text[:p]    

    n = text.find('창닫기')
    if n != -1:
        text = text[:n]
    
    # 마지막으로 희생당한 기사를 위한 코드인데, 간혹 기사의 길이가 너무 짧거나, 기사가 아니거나, 전처리과정에서 희생당한경우( ex)동아닷컴 )
    # 기사의 길이가 현저하게 낮은데 이러한 기사를 ERROR로 append 해주는 함수입니다.
    if len(text) > 100:
        CONTENT_OF_ARTICLES.append(text)
    else:
        CONTENT_OF_ARTICLES.append("ERROR")   
        



In [3]:
TITLE_OF_ARTICLE = []
DATE_OF_ARTICLE = []
CONTENT_OF_ARTICLES = []
LINK_OF_ARTICLE = []
DDOS = True
START_DATE = '2020-12-31'
END_DATE = '2010-01-01'
# 전역변수 설정과 keyword 선언입니다.

keyword = quote('인공지능') #또한 인코딩 오류가 나기때문에 인코딩을 위해 quote함수를 사용하였습니다.
starttime = time.time()
for_DDOS_pagenum=0 # 100페이지마다 쉬기위한 변수이고
while True: # while True로 무한반복 해줍니다. (break되기 전가지)
    start_crawling(keyword,START_DATE, END_DATE,for_DDOS_pagenum) #크롤링 시작하고
    for_DDOS_pagenum += 1 # 100페이지 넘어가면 150초 쉬어줍니다.
    print("쉬는중입니다, DDOS차단 방지")
    time.sleep(150)
    if DDOS == False: # 브레이크 해주는 함수입니다.
        break
    
    
runtime = time.time() - starttime

print(f'끝! 걸린시간 = {runtime}초')
print(f'총 {len(TITLE_OF_ARTICLE)}개의 기사 숫자')

1페이지 완료
2페이지 완료
3페이지 완료
4페이지 완료
5페이지 완료
6페이지 완료
7페이지 완료
8페이지 완료
9페이지 완료
10페이지 완료
11페이지 완료
12페이지 완료
13페이지 완료
14페이지 완료
15페이지 완료
16페이지 완료
17페이지 완료
18페이지 완료
19페이지 완료
20페이지 완료
21페이지 완료
22페이지 완료
23페이지 완료
24페이지 완료
25페이지 완료
26페이지 완료
27페이지 완료
28페이지 완료
29페이지 완료
30페이지 완료
31페이지 완료
32페이지 완료
33페이지 완료
34페이지 완료
35페이지 완료
36페이지 완료
37페이지 완료
38페이지 완료
39페이지 완료
40페이지 완료
41페이지 완료
42페이지 완료
43페이지 완료
44페이지 완료
45페이지 완료
46페이지 완료
47페이지 완료
48페이지 완료
49페이지 완료
50페이지 완료
51페이지 완료
52페이지 완료
53페이지 완료
54페이지 완료
55페이지 완료
56페이지 완료
57페이지 완료
58페이지 완료
59페이지 완료
60페이지 완료
61페이지 완료
62페이지 완료
63페이지 완료
64페이지 완료
65페이지 완료
66페이지 완료
67페이지 완료
68페이지 완료
69페이지 완료
70페이지 완료
71페이지 완료
72페이지 완료
73페이지 완료
74페이지 완료
75페이지 완료
76페이지 완료
77페이지 완료
78페이지 완료
79페이지 완료
80페이지 완료
81페이지 완료
82페이지 완료
83페이지 완료
84페이지 완료
85페이지 완료
86페이지 완료
87페이지 완료
88페이지 완료
89페이지 완료
90페이지 완료
91페이지 완료
92페이지 완료
93페이지 완료
94페이지 완료
95페이지 완료
96페이지 완료
97페이지 완료
98페이지 완료
99페이지 완료
100페이지 완료
쉬는중입니다, DDOS차단 방지
101페이지 완료
102페이지 완료
103페이지 완료
104페이지 완료
105페이지 완료
106페이지 완료
107페이지 완료
108페이지 완료
109페이지 완료


URLError: <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>

In [4]:
df = pd.DataFrame(list(zip(LINK_OF_ARTICLE, TITLE_OF_ARTICLE, CONTENT_OF_ARTICLES, DATE_OF_ARTICLE)), columns =['LINK','Title', 'Content', 'Date'])
df = df[df.Content != "ERROR"]

In [5]:
df['Content'] = df['Content'].str.replace(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)" , '')
df['Content'] = df['Content'].str.replace('[', '')
df['Content'] = df['Content'].str.replace(']', '')
df['Content'] = df['Content'].str.replace("'", "")  
df['Content'] = df['Content'].str.replace('"', '')
df['Content'] = df['Content'].str.replace(',', '')
df['Content'] = df['Content'].str.replace(r'\\n', '') 
df['Content'] = df['Content'].str.replace(r'\\r', '')
df['Content'] = df['Content'].str.replace(r'\\xa0', '')
df['Content'] = df['Content'].str.replace('‘', '')
df['Content'] = df['Content'].str.replace('’', '')
df['Content'] = df['Content'].str.replace('“', '')
df['Content'] = df['Content'].str.replace('”', '')
df['Content'] = df['Content'].str.replace('//', '')
df['Content'] = df['Content'].str.replace('ㆍ', '')
df['Content'] = df['Content'].str.replace('  ', '')
df['Content'] = df['Content'].str.replace('·', '')
df['Content'] = df['Content'].str.replace('…', '')
df['Content'] = df['Content'].str.replace('〈br〉', '')
df['Content'] = df['Content'].str.replace('「', '')
df['Content'] = df['Content'].str.replace('」', '')
df['Content'] = df['Content'].str.replace(r'\\', '')
df['Content'] = df['Content'].str.lstrip()
df['Content'] = df['Content'].str.replace(r'\(([^)]+)\)','')

df['Content'] = df['Content'].str.replace('[.][.]', '.')

df['Title'] = df['Title'].str.replace(r'\[([^)]+)\]','')
df['Title'] = df['Title'].str.replace('‘', '')
df['Title'] = df['Title'].str.replace('’', '')
df['Title'] = df['Title'].str.replace(',', '')
df['Title'] = df['Title'].str.replace('[', '')
df['Title'] = df['Title'].str.replace(']', '')
df['Title'] = df['Title'].str.replace('…', '')
df['Title'] = df['Title'].str.replace('-', '')
df['Title'] = df['Title'].str.replace('·', '')
df['Title'] = df['Title'].str.replace('“', '')
df['Title'] = df['Title'].str.replace('”', '')
df['Title'] = df['Title'].str.replace('"', '')
df['Title'] = df['Title'].str.replace('···', '')
df['Title'] = df['Title'].str.replace("'", '')
df['Title'] = df['Title'].str.replace('?', '')
df['Title'] = df['Title'].str.replace('\n', '')

In [6]:
df.to_csv(f'동아신문1_{keyword}({START_DATE}~{END_DATE}).csv', encoding = 'utf-8-sig', index_label = False, index = False)


 ===========================
# 150페이지부터 차단이 되길래 100페이지마다 쉬어갔는데도, 
# 결국 663페이지째에 차단된 모습입니다. 
# urllib를 사용했음에도 불구하고 시간이 꽤나 오래걸려서 앞의 데이터는 살리고
# 663페이지부터 다시 데이터를 모은뒤 취합하는 과정을 거쳤습니다.

In [7]:
def start_crawling(keyword,START_DATE,END_DATE):#동일합니다.
    URL = f'ttps://www.donga.com/news/search?p=1&query={keyword}&check_news=1&more=1&sorting=1&search_date=1&v1=&v2=&range=1'
    start_date = datetime.datetime.strptime(START_DATE,'%Y-%m-%d')
    end_date = datetime.datetime.strptime(END_DATE,'%Y-%m-%d')
    get_link_from_news_title(URL,start_date,end_date)

def get_link_from_news_title(hURL,START_DATE,END_DATE):
    firstURL = 'h' + hURL
    req = urllib.request.Request(firstURL, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'})        
    source_code_from_url = urllib.request.urlopen(req)
    firstsoup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')
    num1 = firstsoup.select('#content > div.searchContWrap > div.searchCont > h2 > span:nth-child(1)')[0].get_text()
    num2 = num1.replace('(총 ', '')
    num = int(num2.replace(' 건 검색)', ''))
    page_num = int(num /15) +1
    
    Flag = True
    for j in range(663,page_num): # <------- 
        #이부분이 다른데, 앞페이지는 시작부분인 663부터, page_num은 끝까지 지정하였습니다.
        #이후는 동일합니다
        
        
        article_number = (j*15) +1
        URL = f'ttps://www.donga.com/news/search?p={article_number}&query={keyword}&check_news=1&more=1&sorting=1&search_date=1&v1=&v2=&range=1'
        req = urllib.request.Request('h'+URL, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'})        
        source_code_from_url = urllib.request.urlopen(req)
        soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')
        for i in range(0,len(soup.select('p.tit'))):        
            date1 = soup.select(f'#content > div.searchContWrap > div.searchCont > div:nth-child({i+2}) > div.t > p.tit > span')[0]
            date2 = date1.get_text()[:10]
            try:
                date = datetime.datetime.strptime(date2,'%Y-%m-%d')
            except:
                date = lastdate
            lastdate = date            
            if START_DATE >= date:
                if date >= END_DATE:
                    str_date = str(date)[:10]
                    title_name = soup.select(f'#content > div.searchContWrap > div.searchCont > div:nth-child({i+2}) > div.t > p.tit > a:nth-child(1)')[0].get_text()
                    title_link = soup.select(f'#content > div.searchContWrap > div.searchCont > div:nth-child({i+2}) > div.t > p.tit > a:nth-child(1)')[0]['href']
                    TITLE_OF_ARTICLE.append(title_name) 
                    DATE_OF_ARTICLE.append(str_date)
                    LINK_OF_ARTICLE.append(title_link)
                    get_text(title_link)
                else:
                    Flag = False                
                    break 
        
        print(f'{j+1}페이지 완료') 
        
        if Flag == False:
            break

In [1]:
TITLE_OF_ARTICLE = []
DATE_OF_ARTICLE = []
CONTENT_OF_ARTICLES = []
LINK_OF_ARTICLE = []
DDOS = True
START_DATE = '2020-12-31'
END_DATE = '2010-01-01'
keyword = quote('인공지능')

starttime = time.time()

start_crawling(keyword,START_DATE, END_DATE) 

    
    
runtime = time.time() - starttime

print(f'끝! 걸린시간 = {runtime}초')
print(f'총 {len(TITLE_OF_ARTICLE)}개의 기사 숫자')

NameError: name 'quote' is not defined

In [11]:
df = pd.DataFrame(list(zip(LINK_OF_ARTICLE, TITLE_OF_ARTICLE, CONTENT_OF_ARTICLES, DATE_OF_ARTICLE)), columns =['LINK','Title', 'Content', 'Date'])
df = df[df.Content != "ERROR"]

In [12]:
df['Content'] = df['Content'].str.replace(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)" , '')
df['Content'] = df['Content'].str.replace('[', '')
df['Content'] = df['Content'].str.replace(']', '')
df['Content'] = df['Content'].str.replace("'", "")  
df['Content'] = df['Content'].str.replace('"', '')
df['Content'] = df['Content'].str.replace(',', '')
df['Content'] = df['Content'].str.replace(r'\\n', '') 
df['Content'] = df['Content'].str.replace(r'\\r', '')
df['Content'] = df['Content'].str.replace(r'\\xa0', '')
df['Content'] = df['Content'].str.replace('‘', '')
df['Content'] = df['Content'].str.replace('’', '')
df['Content'] = df['Content'].str.replace('“', '')
df['Content'] = df['Content'].str.replace('”', '')
df['Content'] = df['Content'].str.replace('//', '')
df['Content'] = df['Content'].str.replace('ㆍ', '')
df['Content'] = df['Content'].str.replace('  ', '')
df['Content'] = df['Content'].str.replace('·', '')
df['Content'] = df['Content'].str.replace('…', '')
df['Content'] = df['Content'].str.replace('〈br〉', '')
df['Content'] = df['Content'].str.replace('「', '')
df['Content'] = df['Content'].str.replace('」', '')
df['Content'] = df['Content'].str.replace(r'\\', '')
df['Content'] = df['Content'].str.lstrip()
df['Content'] = df['Content'].str.replace(r'\(([^)]+)\)','')

df['Content'] = df['Content'].str.replace('[.][.]', '.')

df['Title'] = df['Title'].str.replace(r'\[([^)]+)\]','')
df['Title'] = df['Title'].str.replace('‘', '')
df['Title'] = df['Title'].str.replace('’', '')
df['Title'] = df['Title'].str.replace(',', '')
df['Title'] = df['Title'].str.replace('[', '')
df['Title'] = df['Title'].str.replace(']', '')
df['Title'] = df['Title'].str.replace('…', '')
df['Title'] = df['Title'].str.replace('-', '')
df['Title'] = df['Title'].str.replace('·', '')
df['Title'] = df['Title'].str.replace('“', '')
df['Title'] = df['Title'].str.replace('”', '')
df['Title'] = df['Title'].str.replace('"', '')
df['Title'] = df['Title'].str.replace('···', '')
df['Title'] = df['Title'].str.replace("'", '')
df['Title'] = df['Title'].str.replace('?', '')
df['Title'] = df['Title'].str.replace('\n', '')

In [13]:
df.to_csv(f'동아신문2_{keyword}({START_DATE}~{END_DATE}).csv', encoding = 'utf-8-sig', index_label = False, index = False)


# 동아신문1.csv와 동아신문2.csv로 따로 저장한 후,
# 잘린부분을 확인한뒤 csv파일 자체에서 통합했습니다.